<a href="https://colab.research.google.com/github/GalinaZh/Appl_alg2021/blob/main/Applied_Alg_sem_8_repete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 8 
# Прикладная алгебра и численные методы
## Повторение: псевдообратная матрица, ортогонализация Грама-Шмидта, LU, QR, SVD, многочлены Чебышева.

In [1]:
!python -m pip install sympy --upgrade

     |████████████████████████████████| 5.9MB 7.5MB/s 
  Found existing installation: sympy 1.1.1
    Uninstalling sympy-1.1.1:
      Successfully uninstalled sympy-1.1.1


In [33]:
import numpy as np
import sympy
from sympy import S, latex, Matrix, I, simplify, expand
import scipy.linalg
import numpy.linalg
from sympy.functions.special.polynomials import chebyshevt, chebyshevu
from numpy.polynomial.chebyshev import chebinterpolate
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

In [ ]:
import sympy
import numpy
sympy.__version__, numpy.__version__

('1.7.1', '1.19.5')

In [8]:
sympy.init_printing(use_unicode=False, wrap_line=False, no_global=True)

## Псевдообратная матрица
В numpy псевдообратная матрица находится с помощью np.linalg.pinv
## Пример 1
Найти псевдообратную матрицу к матрице
$$
\left(
\begin{matrix}
1 & 4 & 7 & 10 & 13\\
16 & 19 & 22 & 25 & 28\\
31 & 34 & 37 & 40 & 43\\
46 & 49 & 52 & 55 & 58
\end{matrix}
\right)
$$

In [28]:
m = 4
n = 5
A = np.arange(1, 3*m*n + 1, 3).reshape((m, n))
As = Matrix(A)
As_pinv = As.pinv()
display(As, As_pinv, As_pinv.evalf(8))
for item in (np.linalg.pinv(A), scipy.linalg.pinv(A)):
    print('A^+ = {0},\n A = A A^+ A  {1}, A^+ = A^+ A ^+ {2}'.format(item, np.allclose(A, np.dot(A, np.dot(item, A))), np.allclose(item, np.dot(item, np.dot(A, item)))))

[1   4   7   10  13]
[                  ]
[16  19  22  25  28]
[                  ]
[31  34  37  40  43]
[                  ]
[46  49  52  55  58]

[-3/50   -7/225  -1/450    2/75 ]
[                               ]
[         -73               23  ]
[-4/125   ----   -1/2250   ---- ]
[         4500             1500 ]
[                               ]
[-1/250  -1/750   1/750   1/250 ]
[                               ]
[          61              -11  ]
[3/125    ----   7/2250    ---- ]
[         4500             1500 ]
[                               ]
[  13      32      11           ]
[ ---     ----    ----    -7/375]
[ 250     1125    2250          ]

[-0.06   -0.031111111   -0.0022222222    0.026666667 ]
[                                                    ]
[-0.032  -0.016222222   -0.00044444444   0.015333333 ]
[                                                    ]
[-0.004  -0.0013333333   0.0013333333       0.004    ]
[                                                    ]
[0.024    0.013555556    0.0031111111   -0.0073333333]
[                                                    ]
[0.052    0.028444444    0.0048888889   -0.018666667 ]

A^+ = [[-0.06       -0.03111111 -0.00222222  0.02666667]
 [-0.032      -0.01622222 -0.00044444  0.01533333]
 [-0.004      -0.00133333  0.00133333  0.004     ]
 [ 0.024       0.01355556  0.00311111 -0.00733333]
 [ 0.052       0.02844444  0.00488889 -0.01866667]],
 A = A A^+ A  True, A^+ = A^+ A ^+ True
A^+ = [[-0.06       -0.03111111 -0.00222222  0.02666667]
 [-0.032      -0.01622222 -0.00044444  0.01533333]
 [-0.004      -0.00133333  0.00133333  0.004     ]
 [ 0.024       0.01355556  0.00311111 -0.00733333]
 [ 0.052       0.02844444  0.00488889 -0.01866667]],
 A = A A^+ A  True, A^+ = A^+ A ^+ True


In [25]:
latex(As)

'\\left[\\begin{matrix}1 & 4 & 7 & 10 & 13\\\\16 & 19 & 22 & 25 & 28\\\\31 & 34 & 37 & 40 & 43\\\\46 & 49 & 52 & 55 & 58\\end{matrix}\\right]'

### LU разложение
$PA = LU$ для матрицы $A$

$L$ - левая треугольная матрица с единицами на главной диагонали,
$U$ - правая треугольная (трапециевидная) матрица,
$P$ - матрица перестановок.

$A = P^{-1}LU$.
### Пример 2.
Построим  LU разложение для матрицы 
$$
M=\left(
\begin{matrix}
0 & 3+ I & 5 - 2I\\
 - I & 2 & 1 - I\\
5 & -1 + 4I & -3
\end{matrix}
\right)
$$

In [35]:
M = Matrix([[0, 3 + I, 5 - 2*I], [-I, 2, 1 - I], [5, -1 + 4*I, -3]])
L, U, perm = M.LUdecomposition()
display(L, U, simplify(expand(L*U)), M, perm)

[ 1           0           0]
[                          ]
[ 0           1           0]
[                          ]
[     (-1 - 6*I)*(3 - I)   ]
[5*I  ------------------  1]
[             10           ]

[-I    2                         1 - I                     ]
[                                                          ]
[0   3 + I                      5 - 2*I                    ]
[                                                          ]
[                              (-1 - 6*I)*(3 - I)*(5 - 2*I)]
[0     0    -3 - 5*I*(1 - I) - ----------------------------]
[                                           10             ]

[-I     2       1 - I ]
[                     ]
[0    3 + I    5 - 2*I]
[                     ]
[5   -1 + 4*I    -3   ]

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

Произведение матриц LU отличается от исходной матрицы M перестановкой строк. Восстановим матрицу M, применяя перестановки в соответствии с результатом, выдаваемым LUdecomposition:

In [37]:
number_of_rows = M.shape[0]
L, U, perm = M.LUdecomposition()
MLU = simplify(expand((L*U).permuteBkwd(perm)))
P = sympy.eye(number_of_rows).permuteFwd(perm)
display('L', L, 'U', U, 'perm', perm, 'P', P, 'MLU', MLU, 
        'M', M, 'P*M == L*U', P*M == simplify(expand(L*U))) 

'L'

[ 1           0           0]
[                          ]
[ 0           1           0]
[                          ]
[     (-1 - 6*I)*(3 - I)   ]
[5*I  ------------------  1]
[             10           ]

'U'

[-I    2                         1 - I                     ]
[                                                          ]
[0   3 + I                      5 - 2*I                    ]
[                                                          ]
[                              (-1 - 6*I)*(3 - I)*(5 - 2*I)]
[0     0    -3 - 5*I*(1 - I) - ----------------------------]
[                                           10             ]

'perm'

...

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

'M'

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

'P*M == L*U'

True

### QR разложение
$A = QR$ 

$Q$ - матрица из ортогональных столбцов, т.е. $Q^HQ = I$, $I$ - единичная матрица, 
$R$ - правая треугольная (трапециевидная) матрица.

Ранг матрицы $A$ равен числу столбцов матрицы $Q$.
### Пример 3.
Построим  QR разложение для матрицы 

$$
M_3 = \left(
\begin{matrix}
0 & 3  & 5  \\
 - 1 & 2 & 1 \\
5 & -1  & -3
\end{matrix}
\right)
$$

In [39]:
M3 = Matrix([[0, 3, 5, 6], [-1, 2, 1, -7], [5, -1, -3, 8]])

Q3, R3 = M3.QRdecomposition()
Q3R3 = simplify(expand(Q3*R3))
display('Q3', simplify(expand(Q3)), 'R3', simplify(expand(R3)), 
        'Q3R3', Q3R3, 'M3', M3, 'M3 = Q3R3', M3 == Q3R3)

'Q3'

[             _____       ____]
[           \/ 910    3*\/ 35 ]
[   0       -------   --------]
[              35        35   ]
[                             ]
[   ____       _____     ____ ]
[-\/ 26    3*\/ 910   -\/ 35  ]
[--------  ---------  --------]
[   26        182        7    ]
[                             ]
[    ____      _____     ____ ]
[5*\/ 26   3*\/ 910   -\/ 35  ]
[--------  ---------  --------]
[   26        910        35   ]

'R3'

[             ____        ____        ____ ]
[  ____  -7*\/ 26    -8*\/ 26    47*\/ 26  ]
[\/ 26   ----------  ----------  --------- ]
[            26          13          26    ]
[                                          ]
[            _____        _____       _____]
[        3*\/ 910    68*\/ 910   15*\/ 910 ]
[  0     ---------   ----------  ----------]
[            26         455         182    ]
[                                          ]
[                         ____        ____ ]
[                    13*\/ 35     9*\/ 35  ]
[  0         0       ---------    -------- ]
[                        35          7     ]

'Q3R3'

[0   3   5   6 ]
[              ]
[-1  2   1   -7]
[              ]
[5   -1  -3  8 ]

'M3'

[0   3   5   6 ]
[              ]
[-1  2   1   -7]
[              ]
[5   -1  -3  8 ]

'M3 = Q3R3'

True